In [ ]:
!nvidia-smi

Tue Oct 21 17:43:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Uninstall bitsandbytes first (causes conflicts)
!pip uninstall -y bitsandbytes

# Install correct versions WITHOUT bitsandbytes
!pip install -q transformers==4.35.2
!pip install -q datasets==2.16.0
!pip install -q accelerate==0.25.0
!pip install -q peft==0.6.0
!pip install -q trl==0.7.0
!pip install -q evaluate
!pip install -q rouge-score
!pip install -q scikit-learn
!pip install -q matplotlib
!pip install -q seaborn

print("✅ Packages installed (without bitsandbytes)")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Navigate to your uploaded folder
os.chdir('/content/drive/MyDrive/ReviewSense-AI')

# Verify you're in the right place
!pwd
!ls

/content/drive/MyDrive/ReviewSense-AI
colab_data.zip


In [ ]:
# Extract the zip file
!unzip -q colab_data.zip

# Verify extraction
!ls data/splits/

test.jsonl  train.jsonl  val.jsonl


In [ ]:
import json
from transformers import AutoTokenizer

# Load your config.json
with open('config.json') as f:
    config = json.load(f)

print(f"✅ Config loaded")
print(f"   Model: {config['base_model']}")

# Load tokenizer
print("\n📥 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(config['base_model'])

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded")
print(f"   Vocabulary size: {len(tokenizer):,}")

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


✅ Config loaded
   Model: TinyLlama/TinyLlama-1.1B-Chat-v1.0

📥 Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

✅ Tokenizer loaded
   Vocabulary size: 32,000


In [ ]:
from datasets import Dataset, DatasetDict

# Load your JSONL files
def load_jsonl(path):
    import json
    data = []
    with open(path) as f:
        for line in f:
            data.append(json.loads(line))
    return data

print("📥 Loading your data...")
train_data = load_jsonl('data/splits/train.jsonl')
val_data = load_jsonl('data/splits/val.jsonl')
test_data = load_jsonl('data/splits/test.jsonl')

print(f"✅ Loaded:")
print(f"   Train: {len(train_data):,} examples")
print(f"   Validation: {len(val_data):,} examples")
print(f"   Test: {len(test_data):,} examples")

# Format for training
def format_example(ex):
    text = f"<s>[INST] {ex['instruction']} [/INST] {ex['response']}</s>"
    return {"text": text}

print("\n🔄 Formatting...")
# Use 5000 for faster training (or use all: len(train_data))
train_formatted = [format_example(ex) for ex in train_data[:5000]]
val_formatted = [format_example(ex) for ex in val_data[:500]]

print(f"✅ Formatted: {len(train_formatted):,} train examples")

# Create HuggingFace datasets
dataset_dict = DatasetDict({
    'train': Dataset.from_list(train_formatted),
    'validation': Dataset.from_list(val_formatted)
})

# Tokenize
def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=1024,
        padding=False,
    )
    result['labels'] = result['input_ids']
    return result

print("\n🔄 Tokenizing (takes 3-5 minutes)...")
tokenized_datasets = dataset_dict.map(
    tokenize_function,
    batched=True,
    remove_columns=['text'],
    desc="Tokenizing"
)

print(f"\n✅ Tokenization complete!")
print(f"   Train: {len(tokenized_datasets['train']):,} examples")
print(f"   Validation: {len(tokenized_datasets['validation']):,} examples")
print(f"   Columns: {tokenized_datasets['train'].column_names}")

📥 Loading your data...
✅ Loaded:
   Train: 14,000 examples
   Validation: 3,000 examples
   Test: 3,000 examples

🔄 Formatting...
✅ Formatted: 5,000 train examples

🔄 Tokenizing (takes 3-5 minutes)...


Tokenizing:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/500 [00:00<?, ? examples/s]


✅ Tokenization complete!
   Train: 5,000 examples
   Validation: 500 examples
   Columns: ['input_ids', 'attention_mask', 'labels']


In [ ]:
from transformers import AutoModelForCausalLM
import torch

print("📥 Loading TinyLlama model...")
print("⏳ First time: Downloads ~2.2GB (takes 1-2 minutes)")

model = AutoModelForCausalLM.from_pretrained(
    config['base_model'],
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

print(f"\n✅ Model loaded successfully!")
print(f"   Parameters: {model.num_parameters():,}")
print(f"   GPU Memory used: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"   GPU Memory available: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1e9:.1f} GB")

📥 Loading TinyLlama model...
⏳ First time: Downloads ~2.2GB (takes 1-2 minutes)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


✅ Model loaded successfully!
   Parameters: 1,100,048,384
   GPU Memory used: 2.21 GB
   GPU Memory available: 13.6 GB


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

print("🔧 Applying LoRA (no quantization)...")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

print("\n✅ LoRA applied successfully!")

🔧 Applying LoRA (no quantization)...
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.20437245579516677

✅ LoRA applied successfully!


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
from datetime import datetime
import numpy as np
import json
import torch
import gc

print("="*80)
print("🚀 EXPERIMENT 1: BASELINE (LR=2e-4, r=8)")
print("="*80)

# Clear memory
gc.collect()
torch.cuda.empty_cache()

# Load model
model = AutoModelForCausalLM.from_pretrained(
    config['base_model'],
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

# Apply LoRA
lora_cfg_exp1 = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_cfg_exp1)
model.print_trainable_parameters()

print(f"\nGPU Memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Training config
training_args = TrainingArguments(
    output_dir="./models/exp1",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.03,
    logging_steps=25,
    evaluation_strategy="steps",
    eval_steps=250,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    optim="adamw_torch",
    report_to="none",
    seed=42,
)

collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=collator
)

print(f"\n⏰ Starting: {datetime.now().strftime('%H:%M:%S')}")
print("⏱️  Estimated: 45-60 minutes")
print("="*80 + "\n")

start_time_exp1 = datetime.now()
result_exp1 = trainer.train()
end_time_exp1 = datetime.now()

duration_exp1 = (end_time_exp1 - start_time_exp1).total_seconds() / 60

print(f"\n{'='*80}")
print("✅ EXPERIMENT 1 COMPLETE!")
print("="*80)
print(f"   Train Loss: {result_exp1.metrics['train_loss']:.4f}")
print(f"   Runtime: {duration_exp1:.1f} minutes")

# Save model
trainer.save_model("./models/exp1")
trainer.save_metrics("train", result_exp1.metrics)

# Evaluate
eval_exp1 = trainer.evaluate()
print(f"   Eval Loss: {eval_exp1['eval_loss']:.4f}")
print(f"   Perplexity: {np.exp(eval_exp1['eval_loss']):.2f}")

trainer.save_metrics("eval", eval_exp1)

# Save experiment info
exp1_info = {
    "experiment": "Experiment 1 - Baseline",
    "learning_rate": 2e-4,
    "lora_r": 8,
    "lora_alpha": 16,
    "batch_size": 2,
    "train_loss": float(result_exp1.metrics['train_loss']),
    "eval_loss": float(eval_exp1['eval_loss']),
    "perplexity": float(np.exp(eval_exp1['eval_loss'])),
    "runtime_minutes": float(duration_exp1),
    "completed_at": datetime.now().isoformat()
}

with open("./models/exp1/experiment_info.json", 'w') as f:
    json.dump(exp1_info, f, indent=2)

print(f"\n💾 Saved to: ./models/exp1/")
print("="*80)

🚀 EXPERIMENT 1: BASELINE (LR=2e-4, r=8)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.20437245579516677

GPU Memory: 2.22 GB

⏰ Starting: 17:55:57
⏱️  Estimated: 45-60 minutes



/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
250,0.711200,0.707322
500,0.736900,0.697550
750,0.704100,0.693045
1000,0.730100,0.686831
1250,0.722300,0.684579
1500,0.716800,0.683102
1750,0.658000,0.681559



✅ EXPERIMENT 1 COMPLETE!
   Train Loss: 0.7403
   Runtime: 48.3 minutes


   Eval Loss: 0.6810
   Perplexity: 1.98

💾 Saved to: ./models/exp1/


In [ ]:
print("\n" + "="*80)
print("🚀 EXPERIMENT 2: LOWER LEARNING RATE (LR=1e-4, r=8)")
print("="*80)

# Clear memory thoroughly
del model, trainer, collator
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

print("🧹 GPU memory cleared")
print(f"   Available: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1e9:.1f} GB")

# Reload fresh model
model = AutoModelForCausalLM.from_pretrained(
    config['base_model'],
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

# Apply same LoRA config as Exp1
lora_cfg_exp2 = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_cfg_exp2)
model.print_trainable_parameters()

print(f"\nGPU Memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Training config with LOWER learning rate
training_args = TrainingArguments(
    output_dir="./models/exp2",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,  # LOWER: Half of baseline
    weight_decay=0.01,
    warmup_ratio=0.03,
    logging_steps=25,
    evaluation_strategy="steps",
    eval_steps=250,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    optim="adamw_torch",
    report_to="none",
    seed=42,
)

collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=collator
)

print(f"\n⏰ Starting: {datetime.now().strftime('%H:%M:%S')}")
print("⏱️  Estimated: 45-60 minutes")
print("="*80 + "\n")

start_time_exp2 = datetime.now()
result_exp2 = trainer.train()
end_time_exp2 = datetime.now()

duration_exp2 = (end_time_exp2 - start_time_exp2).total_seconds() / 60

print(f"\n{'='*80}")
print("✅ EXPERIMENT 2 COMPLETE!")
print("="*80)
print(f"   Train Loss: {result_exp2.metrics['train_loss']:.4f}")
print(f"   Runtime: {duration_exp2:.1f} minutes")

# Save
trainer.save_model("./models/exp2")
trainer.save_metrics("train", result_exp2.metrics)

eval_exp2 = trainer.evaluate()
print(f"   Eval Loss: {eval_exp2['eval_loss']:.4f}")
print(f"   Perplexity: {np.exp(eval_exp2['eval_loss']):.2f}")

trainer.save_metrics("eval", eval_exp2)

exp2_info = {
    "experiment": "Experiment 2 - Lower LR",
    "learning_rate": 1e-4,
    "lora_r": 8,
    "lora_alpha": 16,
    "batch_size": 2,
    "train_loss": float(result_exp2.metrics['train_loss']),
    "eval_loss": float(eval_exp2['eval_loss']),
    "perplexity": float(np.exp(eval_exp2['eval_loss'])),
    "runtime_minutes": float(duration_exp2),
    "completed_at": datetime.now().isoformat()
}

with open("./models/exp2/experiment_info.json", 'w') as f:
    json.dump(exp2_info, f, indent=2)

print(f"\n💾 Saved to: ./models/exp2/")
print("="*80)


🚀 EXPERIMENT 2: LOWER LEARNING RATE (LR=1e-4, r=8)
🧹 GPU memory cleared
   Available: 13.6 GB


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.20437245579516677

GPU Memory: 4.46 GB

⏰ Starting: 18:45:46
⏱️  Estimated: 45-60 minutes



/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
250,0.723600,0.719495
500,0.743800,0.703421
750,0.713500,0.696506
1000,0.739800,0.693653
1250,0.732800,0.689961
1500,0.733000,0.688446
1750,0.673100,0.687195



✅ EXPERIMENT 2 COMPLETE!
   Train Loss: 0.7623
   Runtime: 47.6 minutes


   Eval Loss: 0.6868
   Perplexity: 1.99

💾 Saved to: ./models/exp2/


In [ ]:
print("\n" + "="*80)
print("🚀 EXPERIMENT 3: HIGHER LoRA RANK (LR=2e-4, r=16)")
print("="*80)

# AGGRESSIVE memory cleanup
del model, trainer, collator
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

print("🧹 GPU memory cleared aggressively")

# Check available memory
available_mem = (torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1e9
print(f"   Available GPU memory: {available_mem:.1f} GB")

if available_mem < 10:
    print("⚠️  Low memory - may need to reduce batch size further")

# Reload model
model = AutoModelForCausalLM.from_pretrained(
    config['base_model'],
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

# Higher rank LoRA configuration
lora_cfg_exp3 = LoraConfig(
    r=16,  # HIGHER rank (2x baseline)
    lora_alpha=32,  # Alpha = 2 * r
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_cfg_exp3)
model.print_trainable_parameters()

print(f"\nGPU Memory after LoRA: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Training config - REDUCED batch size for r=16
training_args = TrainingArguments(
    output_dir="./models/exp3",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # REDUCED to 1 (r=16 uses more memory)
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # Increased to keep effective batch=8
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.03,
    max_grad_norm=0.3,
    logging_steps=25,
    evaluation_strategy="steps",
    eval_steps=250,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    optim="adamw_torch",
    report_to="none",
    seed=42,
)

collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=collator
)

print(f"\n⏰ Starting: {datetime.now().strftime('%H:%M:%S')}")
print("⏱️  Estimated: 1.5-2 hours (slower due to batch=1)")
print("="*80 + "\n")

start_time_exp3 = datetime.now()

try:
    result_exp3 = trainer.train()
    end_time_exp3 = datetime.now()

    duration_exp3 = (end_time_exp3 - start_time_exp3).total_seconds() / 60

    print(f"\n{'='*80}")
    print("✅ EXPERIMENT 3 COMPLETE!")
    print("="*80)
    print(f"   Train Loss: {result_exp3.metrics['train_loss']:.4f}")
    print(f"   Runtime: {duration_exp3:.1f} minutes")

    trainer.save_model("./models/exp3")
    trainer.save_metrics("train", result_exp3.metrics)

    eval_exp3 = trainer.evaluate()
    print(f"   Eval Loss: {eval_exp3['eval_loss']:.4f}")
    print(f"   Perplexity: {np.exp(eval_exp3['eval_loss']):.2f}")

    trainer.save_metrics("eval", eval_exp3)

    exp3_info = {
        "experiment": "Experiment 3 - Higher Rank",
        "learning_rate": 2e-4,
        "lora_r": 16,
        "lora_alpha": 32,
        "batch_size": 1,
        "train_loss": float(result_exp3.metrics['train_loss']),
        "eval_loss": float(eval_exp3['eval_loss']),
        "perplexity": float(np.exp(eval_exp3['eval_loss'])),
        "runtime_minutes": float(duration_exp3),
        "completed_at": datetime.now().isoformat()
    }

    with open("./models/exp3/experiment_info.json", 'w') as f:
        json.dump(exp3_info, f, indent=2)

    print(f"\n💾 Saved to: ./models/exp3/")
    print("="*80)
    print("\n🎉 EXPERIMENT 3 SUCCESS!")

except torch.cuda.OutOfMemoryError as e:
    print(f"\n❌ Out of memory error: {e}")
    print("\n💡 This is okay - you already have 2 successful experiments!")
    print("   2 experiments is sufficient for assignment requirements")

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


🚀 EXPERIMENT 3: HIGHER LoRA RANK (LR=2e-4, r=16)
🧹 GPU memory cleared aggressively
   Available GPU memory: 11.4 GB


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.40791125334440875

GPU Memory after LoRA: 6.69 GB

⏰ Starting: 19:46:58
⏱️  Estimated: 1.5-2 hours (slower due to batch=1)



/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
250,0.724000,0.727263
500,0.752200,0.718609
750,0.696500,0.714720
1000,0.724700,0.709675
1250,0.726500,0.706571
1500,0.708400,0.707844
1750,0.645100,0.706550



✅ EXPERIMENT 3 COMPLETE!
   Train Loss: 0.7369
   Runtime: 50.1 minutes


   Eval Loss: 0.7062
   Perplexity: 2.03

💾 Saved to: ./models/exp3/

🎉 EXPERIMENT 3 SUCCESS!


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os

print("="*80)
print("📊 EXPERIMENT COMPARISON & ANALYSIS")
print("="*80)

# Load all experiment results
experiments = []

# Load Exp1
if os.path.exists("./models/exp1/experiment_info.json"):
    with open("./models/exp1/experiment_info.json") as f:
        experiments.append(json.load(f))
    print("✅ Loaded Experiment 1")

# Load Exp2
if os.path.exists("./models/exp2/experiment_info.json"):
    with open("./models/exp2/experiment_info.json") as f:
        experiments.append(json.load(f))
    print("✅ Loaded Experiment 2")

# Load Exp3
if os.path.exists("./models/exp3/experiment_info.json"):
    with open("./models/exp3/experiment_info.json") as f:
        experiments.append(json.load(f))
    print("✅ Loaded Experiment 3")
else:
    print("⚠️  Experiment 3 not found (memory issue - this is okay)")

print(f"\n📊 Total experiments completed: {len(experiments)}")

if len(experiments) == 0:
    print("❌ No experiments found! Check if training completed.")
else:
    # Create DataFrame
    df = pd.DataFrame(experiments)

    print("\n" + "="*80)
    print("📋 RESULTS TABLE")
    print("="*80)
    print(df[['experiment', 'learning_rate', 'lora_r', 'train_loss', 'eval_loss', 'perplexity', 'runtime_minutes']].to_string(index=False))

    # Find best model
    best_idx = df['eval_loss'].idxmin()
    best_model = df.iloc[best_idx]

    print("\n" + "="*80)
    print("🏆 BEST MODEL")
    print("="*80)
    print(f"   Experiment: {best_model['experiment']}")
    print(f"   Learning Rate: {best_model['learning_rate']}")
    print(f"   LoRA Rank: {best_model['lora_r']}")
    print(f"   Train Loss: {best_model['train_loss']:.4f}")
    print(f"   Eval Loss: {best_model['eval_loss']:.4f}")
    print(f"   Perplexity: {best_model['perplexity']:.2f}")
    print(f"   Runtime: {best_model['runtime_minutes']:.1f} minutes")

    # Calculate total time
    total_time = df['runtime_minutes'].sum()
    print(f"\n⏰ Total Training Time: {total_time:.1f} minutes ({total_time/60:.2f} hours)")

    # Save comparison
    df.to_csv('experiment_comparison.csv', index=False)
    print("\n✅ Results saved to: experiment_comparison.csv")

    # Create visualizations
    print("\n📊 Creating comparison charts...")

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

    # Plot 1: Training Loss
    axes[0, 0].bar(range(len(df)), df['train_loss'], color=colors[:len(df)])
    axes[0, 0].set_title('Training Loss Comparison', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Experiment')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_xticks(range(len(df)))
    axes[0, 0].set_xticklabels([f"Exp {i+1}" for i in range(len(df))])
    axes[0, 0].grid(axis='y', alpha=0.3)
    for i, v in enumerate(df['train_loss']):
        axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', fontsize=10)

    # Plot 2: Evaluation Loss
    axes[0, 1].bar(range(len(df)), df['eval_loss'], color=colors[:len(df)])
    axes[0, 1].set_title('Evaluation Loss Comparison', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Experiment')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_xticks(range(len(df)))
    axes[0, 1].set_xticklabels([f"Exp {i+1}" for i in range(len(df))])
    axes[0, 1].grid(axis='y', alpha=0.3)
    for i, v in enumerate(df['eval_loss']):
        axes[0, 1].text(i, v + 0.01, f'{v:.3f}', ha='center', fontsize=10)

    # Plot 3: Perplexity
    axes[1, 0].bar(range(len(df)), df['perplexity'], color=colors[:len(df)])
    axes[1, 0].set_title('Perplexity Comparison (Lower is Better)', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Experiment')
    axes[1, 0].set_ylabel('Perplexity')
    axes[1, 0].set_xticks(range(len(df)))
    axes[1, 0].set_xticklabels([f"Exp {i+1}" for i in range(len(df))])
    axes[1, 0].grid(axis='y', alpha=0.3)
    for i, v in enumerate(df['perplexity']):
        axes[1, 0].text(i, v + 0.02, f'{v:.2f}', ha='center', fontsize=10)

    # Plot 4: Runtime
    axes[1, 1].bar(range(len(df)), df['runtime_minutes'], color=colors[:len(df)])
    axes[1, 1].set_title('Training Time Comparison', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Experiment')
    axes[1, 1].set_ylabel('Minutes')
    axes[1, 1].set_xticks(range(len(df)))
    axes[1, 1].set_xticklabels([f"Exp {i+1}" for i in range(len(df))])
    axes[1, 1].grid(axis='y', alpha=0.3)
    for i, v in enumerate(df['runtime_minutes']):
        axes[1, 1].text(i, v + 1, f'{v:.0f}min', ha='center', fontsize=10)

    plt.tight_layout()
    plt.savefig('experiment_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✅ Comparison charts created and saved")

    # Save comprehensive summary
    summary = {
        "project": "ReviewSense AI - Product Review Intelligence",
        "student": "Sravan Kumar Kurapati",
        "university": "Northeastern University",
        "base_model": config['base_model'],
        "total_experiments": len(experiments),
        "total_training_hours": total_time / 60,
        "best_model": {
            "experiment": best_model['experiment'],
            "eval_loss": float(best_model['eval_loss']),
            "perplexity": float(best_model['perplexity']),
            "learning_rate": float(best_model['learning_rate']),
            "lora_rank": int(best_model['lora_r'])
        },
        "all_experiments": experiments,
        "completed_at": datetime.now().isoformat()
    }

    with open('training_summary.json', 'w') as f:
        json.dump(summary, f, indent=2)

    print("\n✅ Summary saved to: training_summary.json")

    print("\n" + "="*80)
    print("🎉 HYPERPARAMETER OPTIMIZATION COMPLETE!")
    print("="*80)
    print(f"\n✅ Tested {len(experiments)} configurations:")
    print(f"   1. Baseline: LR=2e-4, r=8")
    print(f"   2. Lower LR: LR=1e-4, r=8")
    if len(experiments) == 3:
        print(f"   3. Higher Rank: LR=2e-4, r=16")

    print(f"\n🏆 Winner: {best_model['experiment']}")
    print(f"   Improvement over worst: {(df['eval_loss'].max() - best_model['eval_loss']) / df['eval_loss'].max() * 100:.1f}%")

In [ ]:
# Create zip of all models and results
!zip -r all_training_results.zip models/ *.csv *.png *.json

# Check file size
!ls -lh all_training_results.zip

# Download
from google.colab import files

print("📥 Downloading all results...")
print("   File size: ~500MB-1GB")
print("   This may take 3-5 minutes")

files.download('all_training_results.zip')

print("\n✅ Download complete!")
print("\n📁 Files also available in Google Drive:")
print("   /MyDrive/ReviewSense-AI/models/")

In [ ]:
print("="*80)
print("🎯 ASSIGNMENT COMPLETION STATUS")
print("="*80)

completed = {
    "✅ Dataset Preparation": "12/12 points",
    "✅ Model Selection": "10/10 points",
    "✅ Fine-Tuning Setup": "12/12 points",
    "✅ Hyperparameter Optimization": "10/10 points",
}

remaining = {
    "⏳ Model Evaluation": "12 points (next)",
    "⏳ Error Analysis": "8 points",
    "⏳ Inference Pipeline": "6 points",
    "⏳ Video & Documentation": "10 points",
}

print("\n✅ COMPLETED (44/80 functional points):")
for item, points in completed.items():
    print(f"   {item}: {points}")

print("\n⏳ REMAINING (36/80 functional points):")
for item, points in remaining.items():
    print(f"   {item}: {points}")

print("\n🎯 CURRENT SCORE: 44/100")
print("🎯 TARGET: 100/100")

print("\n📋 NEXT STEPS (On Your Mac):")
print("   1. Download all_training_results.zip")
print("   2. Extract models to Mac project folder")
print("   3. Run comprehensive evaluation (ROUGE, BLEU, custom metrics)")
print("   4. Perform error analysis (identify failure patterns)")
print("   5. Build Streamlit demo application")
print("   6. Write technical report (5-7 pages)")
print("   7. Record video walkthrough (5-10 minutes)")

print("\n⏰ Estimated remaining work: 2-3 days")
print("🎉 You've completed the hardest part - training!")
print("="*80)